# Pretraining a Born Classifier

We'd like to extract as much performance form Born as possible, without complicating the model (we've seen that ensembling is not particularly effective anyhow, see the `born_ensemble.ipynb` notebook).

Pretraining seems like a natural way to go about it!

In [1]:
import os
import pandas as pd

from utils import prepare_data

In [2]:
datadir = "data"

In [4]:
df = pd.read_parquet(os.path.join(datadir, "pretrain.parquet"))

processed_df = prepare_data(df, num_processes=5)

Processing pretraining data: 100%|██████████| 2000/2000 [00:19<00:00, 103.94it/s] 


In [8]:
processed_df.sample(15)

,sentence_id,token,pos,dep,ner_tag
70256,0,ИТК,PROPN,npadvmod,ORG
77761,3,Love,PROPN,conj,NONE
177114,6,constant,ADJ,amod,NONE
16142,3,of,ADP,prep,NONE
56492,0,pentafluoride,NOUN,appos,NONE
103322,3,.,PUNCT,punct,NONE
79576,12,upper,ADJ,amod,NONE
184062,0,region,NOUN,pobj,NONE
97702,0,",",PUNCT,punct,DATE
178382,1,",",PUNCT,punct,NONE
